Для выполненения задания в качестве текстов были взяты региональные новости Челябинска, 17 текстов, около 3 тысяч токенов. Первоначальная разметка ключевых слов скорее представляла собой тематические теги, поэтому она была изменена, я удалила те слова, которые не использовались в тексте новости, но описывали его тему. Кроме того, теги были достаточно общими, поэтому я также добавляла дополнительные ключевые слова, например, при новости об открытии центра трансплантации костного мозга в больнице изначально были теги: "больница", "медицина", "ЧОКБ". Я добавила также теги "трансплантация костного мозга" и "пациенты"

In [201]:
import pandas as pd
texts = pd.read_csv('texts.csv', sep = ';')
texts.head()

,texts,key words
0,Согласно исследованиям онлайн-сервиса рекрутин...,"любовь,работа,опрос"
1,17 ноября в мире отмечают День черных кошек. 1...,"Карен Даллакян,черный кот,кот"
2,"Челябинский археолог Евгений Шиманский, которы...","верблюд,Озерск"
3,Постановка Челябинского театра кукол имени Вол...,"премия,куклы,театр,конкурс"
4,В Челябинской области выбрали три лучших школь...,"дети,столовая,школа"


Сделаем препроцессинг текстов:

In [203]:
import RAKE
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t.isalpha():
            lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)
lemm_texts = []
for text in texts['texts'].values:
    lemm_texts.append(normalize_text(text))

### Rake

In [204]:
keywords_rake = []
for text in lemm_texts:
    keywords = rake.run(text, maxWords=3, minFrequency=2)
    only_keywords = []
    for kw in keywords:
        keyword = kw[0]
        only_keywords.append(keyword)
    keywords_rake.append(only_keywords)

### TextRank

In [205]:
from summa import keywords
keywords_tr = []
for text in lemm_texts:
    kw = keywords.keywords(text, language='russian', additional_stopwords=stop, scores = True)
    top_3 = kw[:3]
    only_keywords = []
    for kw in top_3:
        keyword = kw[0]
        only_keywords.append(keyword)
    keywords_tr.append(only_keywords)

### Tf-idf

In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf_vec = TfidfVectorizer(stop_words=stop)
tf_idf = tfidf_vec.fit_transform(lemm_texts).todense()
kw_ind = np.argsort(tf_idf, axis = 1)[:,-3:]
keywords_tf = []
vocab = tfidf_vec.vocabulary_
for ind in kw_ind:
    keywords = []
    for word in vocab:
        if vocab[word] in ind:
            keywords.append(word)
    keywords_tf.append(keywords)

Чтобы посчитать качество, нормализуем эталонные ключевые слова:

In [207]:
in_kws = []
keywords = texts['key words'].values.tolist()
for kws in keywords:
    kws = kws.split(',')
    lemm_kws = []
    for kw in kws:
        parts = kw.split(' ')
        lemmas = []
        for part in parts:
            lemmas.append(m.parse(part)[0].normal_form)
        lemm_kws.append(' '.join(lemmas))
    in_kws.append(lemm_kws)

In [208]:
def count_metrics(kws):
    precisions = []
    recalls = []
    f_scores = []
    for ind, kw in enumerate(kws):
        if len(kw) != 0:
            in_kw = in_kws[ind]
            right_kw = set(kw) & set(in_kw)
            precision = len(right_kw) / len(kw)
            recall = len(right_kw) / len(in_kw)
            if precision != 0 and recall != 0:
                f_score = 2 * (precision * recall) / (precision + recall)
            else:
                f_score = 0
        else:
            precision = 0
            recall = 0
            f_score = 0
        precisions.append(precision)
        recalls.append(recall)
        f_scores.append(f_score)
    t_precision = sum(precisions) / len(precisions)
    t_recall = sum(recalls) / len(recalls)
    t_f_scores = sum(f_scores) / len(f_scores)
    return t_precision, t_recall, t_f_scores

Наилучшее качество у tf-idf:

In [211]:
tf_pr, tf_rec, tf_fscore = count_metrics(keywords_tf)
tr_pr, tr_rec, tr_fscore = count_metrics(keywords_tr)
rake_pr, rake_rec, rake_fscore = count_metrics(keywords_rake)
print(f'Tf-idf precision: {tf_pr} Tf-idf recall: {tf_rec} TF-idf f-score: {tf_fscore}')
print(f'TextRank precision: {tr_pr} TextRank recall: {tr_rec} TextRank f-score: {tr_fscore}')
print(f'Rake precision: {rake_pr} Rake recall: {rake_rec} Rake f-score: {rake_fscore}')

Tf-idf precision: 0.3529411764705882 Tf-idf recall: 0.3205882352941176 TF-idf f-score: 0.3323529411764706
TextRank precision: 0.17647058823529413 TextRank recall: 0.1715686274509804 TextRank f-score: 0.17198879551820728
Rake precision: 0.14705882352941177 Rake recall: 0.053921568627450976 Rake f-score: 0.0784313725490196


Посчитаем качество, отфильтровав нужные ключевые слова с помощью фильтров:

In [213]:
from summa import keywords
keywords_tr = []
for text in lemm_texts:
    kws = keywords.keywords(text, language='russian', additional_stopwords=stop, scores = True)
    fil_kws = []
    for kw in kws:
        parts = kw[0].split(' ')
        pos = []
        for part in parts:
            pos.append(m.parse(part)[0].tag.POS)
        if pos == ['NOUN'] or pos == ['ADJF', 'NOUN'] or pos == ['NOUN', 'ADJF', 'NOUN']:
            fil_kws.append(kw[0])
    keywords_tr.append(fil_kws[:3])

In [214]:
keywords_rake = []
for text in lemm_texts:
    keywords = rake.run(text, maxWords=3, minFrequency=2)
    fil_kws = []
    for kw in keywords:
        parts = kw[0].split(' ')
        pos = []
        for part in parts:
            pos.append(m.parse(part)[0].tag.POS)
        if pos == ['NOUN'] or pos == ['ADJF', 'NOUN']:
            fil_kws.append(kw[0])
    keywords_rake.append(fil_kws)

In [215]:
kw_ind = np.argsort(tf_idf, axis = 1)
keywords_tf = []
words = tfidf_vec.get_feature_names_out()
for row in kw_ind:
    keywords = []
    kws = words[row.tolist()]
    for kw in kws[0]:
        if m.parse(kw)[0].tag.POS == 'NOUN':
            keywords.append(kw)
    keywords_tf.append(keywords[-3:])

Качество немного выросло для всех трех способов:

In [216]:
tf_pr, tf_rec, tf_fscore = count_metrics(keywords_tf)
tr_pr, tr_rec, tr_fscore = count_metrics(keywords_tr)
rake_pr, rake_rec, rake_fscore = count_metrics(keywords_rake)
print(f'Tf-idf precision: {tf_pr} Tf-idf recall: {tf_rec} TF-idf f-score: {tf_fscore}')
print(f'TextRank precision: {tr_pr} TextRank recall: {tr_rec} TextRank f-score: {tr_fscore}')
print(f'Rake precision: {rake_pr} Rake recall: {rake_rec} Rake f-score: {rake_fscore}')

Tf-idf precision: 0.392156862745098 Tf-idf recall: 0.3598039215686274 TF-idf f-score: 0.37156862745098035
TextRank precision: 0.18627450980392157 TextRank recall: 0.1715686274509804 TextRank f-score: 0.17787114845938376
Rake precision: 0.17647058823529413 Rake recall: 0.053921568627450976 Rake f-score: 0.08235294117647059
